# MSc Thesis 
# Jacotte Monroe

Code that retrieves automatically the MODIS scenes for each day of elephant steps. 

## Importing and initializing 

In [1]:
import ee
import geemap
import os
import pandas as pd

ee.Initialize()

## Set-up

In [2]:
# load Etosha National Park study area
enp = ee.FeatureCollection('WCMC/WDPA/current/polygons') \
        .filter(ee.Filter.eq('ORIG_NAME', 'Etosha'))

# turn ENP study area into a geometry
enp_geom = enp.geometry()

In [30]:
# load extent look up table 
extents_lut = pd.read_csv('data/step_extents/LA2_step_ex_w2027.csv')

In [31]:
# get output folder paths for current MODIS images and images from week prior
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
out_dir = os.path.join(os.path.expanduser('~'), 'Documents/MSc_Thesis/data/modis_ssf')
out_dir_lag = os.path.join(os.path.expanduser('~'), 'Documents/MSc_Thesis/data/modis_ssf/lag')

## Functions

In [7]:
# function to clip image to study area
def clipToAOI(image): 
    result = image.clip(bbox)
    return result.copyProperties(image, ['system:id'])

In [8]:
# function to reproject (32733 if want Namibia coordinates, but elephant fixes are in 4326)
def reprojectModis(image):
    return image.reproject('EPSG:4326', None, 250)


## Visualization Parameters

In [11]:
# define visualization parameters
visualization_modis = {
  'min': -100.0,
  'max': 8000.0,
  'bands': ['sur_refl_b02', 'sur_refl_b02', 'sur_refl_b01'],
}

In [23]:
# initialization and set up of the map
Map = geemap.Map()
Map.centerObject(enp_geom)
Map.add_layer(enp_geom, None, 'Etosha National Park')
#Map

## Loop

In [32]:
# loop for all table entries 
# 1. get dates
# 2. get coordinates 
# 3. turn into geometry
# 4. get MODIS image
# 5. clip and reproject 
# 6. visualize

for i in range(len(extents_lut)):
    # select entry
    extent = extents_lut.iloc[i]
    
    # retrieve extent dates 
    start_date = extent.loc['start_date']
    end_date = extent.loc['end_date']
    
    start_date_prev_week = extent.loc['start_date_prev_week']
    end_date_prev_week = extent.loc['end_date_prev_week']
    
    # retrieve extent coordinates 
    # source: https://sparkbyexamples.com/pandas/pandas-get-cell-value-from-dataframe/?utm_content=cmp-true
    # source: https://stackoverflow.com/questions/75203044/how-to-create-polygon-from-bbox-data-in-pythonv
    coords = [[[extent.loc['xmin'], extent.loc['ymin']],
              [extent.loc['xmin'], extent.loc['ymax']],
              [extent.loc['xmax'], extent.loc['ymax']],
              [extent.loc['xmax'], extent.loc['ymin']]]]

    # create geometry from coordinates
    bbox = ee.Geometry.Polygon(coords, None)
    
    # retrieve MODIS 250m image for geometry
    modis = ee.ImageCollection('MODIS/061/MOD09GQ') \
        .filterDate(start_date, end_date) \
        .filterBounds(bbox) \
        .select(['sur_refl_b02', 'sur_refl_b01']) 
    
    modis_prev_week = ee.ImageCollection('MODIS/061/MOD09GQ') \
        .filterDate(start_date_prev_week, end_date_prev_week) \
        .filterBounds(bbox) \
        .select(['sur_refl_b02', 'sur_refl_b01']) 
    
    # reproject image
    modis_reproj = modis.map(reprojectModis)
    
    modis_prev_week_reproj = modis_prev_week.map(reprojectModis)
    
    # clip image
    modis_clipped = modis_reproj.map(clipToAOI)
    
    modis_prev_week_clipped = modis_prev_week_reproj.map(clipToAOI)
    
    # map image
    #Map.add_layer(modis_clipped, visualization_modis, 'MODIS image')
    
    # export MODIS image to local repository 
    # source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
    geemap.ee_export_image_collection(modis_clipped, out_dir = out_dir)
    
    geemap.ee_export_image_collection(modis_prev_week_clipped, out_dir = out_dir_lag)

Total number of images: 1

Exporting 1/1: /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/2008_11_06.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/2008_11_06.tif


Total number of images: 1

Exporting 1/1: /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/lag/2008_10_30.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/lag/2008_10_30.tif


Total number of images: 1

Exporting 1/1: /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/2008_11_07.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/2008_11_07.tif


Total number of images: 1

Exporting 1/1: /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/lag/2008_10_31.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis_ssf/lag/2008_10_31.tif


Total number of images: 1

Exporting 1/1: /home/osboxes/Documents/MSc_Th